In [38]:
# Імпорт необхідних бібліотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import skew

In [39]:
# Кастомний трансформер для зменшення скошеності числових ознак
class SkewnessTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, skew_threshold=0.75):
        self.skew_threshold = skew_threshold

    def fit(self, X, y=None):
        self.skewed_cols_ = []
        # Якщо X є DataFrame
        if hasattr(X, 'columns'):
            for col in X.columns:
                if np.issubdtype(X[col].dtype, np.number):
                    if abs(X[col].skew()) > self.skew_threshold:
                        self.skewed_cols_.append(col)
        else:
            # Якщо X є numpy array
            for i in range(X.shape[1]):
                if abs(skew(X[:, i])) > self.skew_threshold:
                    self.skewed_cols_.append(i)
        return self

    def transform(self, X):
        X_trans = X.copy()
        if hasattr(X, 'columns'):
            for col in self.skewed_cols_:
                X_trans[col] = np.log1p(X_trans[col])
        else:
            for i in self.skewed_cols_:
                X_trans[:, i] = np.log1p(X_trans[:, i])
        return X_trans

In [40]:
# Завантаження даних
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

In [41]:
# Збереження Id для тестових даних та видалення непотрібного стовпця
test_ids = test['Id']
train.drop(columns=['Id'], inplace=True)
test.drop(columns=['Id'], inplace=True)

In [ ]:
# Визначення ознак

In [42]:
# Цільова змінна – логарифмована для нормалізації розподілу
y = np.log1p(train['SalePrice'])
X = train.drop(columns=['SalePrice'])

In [43]:
# Розподіл ознак за типами
num_features = X.select_dtypes(include=['int64', 'float64']).columns.to_list()
cat_features = X.select_dtypes(include=['object']).columns.to_list()

In [ ]:
# Побудова пайплайнів для передобробки даних

In [44]:
# Пайплайн для числових ознак: заповнення пропущених значень, корекція скошеності та масштабування
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('skew', SkewnessTransformer(skew_threshold=0.75)),
    ('scaler', StandardScaler())
])

In [45]:
# Пайплайн для категорійних ознак: заповнення пропущених значень та one-hot кодування
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [46]:
# Комбінування обробників ознак
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])

In [ ]:
# Визначення ансамблевої моделі

In [47]:
# Базові моделі
estimators = [
    ('ridge', Ridge(alpha=1.0)),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('gbr', GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42))
]

In [48]:
# Ансамбль із використанням stacking (передбачено passthrough для передачі вихідних ознак фінальному оцінювачу)
stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=Ridge(alpha=1.0),
    cv=5,
    passthrough=True
)

In [49]:
# Об’єднання передобробки та регресора в єдиний пайплайн
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', stacking_regressor)
])

In [50]:
# **Розподіл даних на тренувальну та валідаційну вибірки**
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
# Навчання моделі
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('skew',
                                                                   SkewnessTransformer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF'...
                                                   'BsmtCond', 'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', ...])])),
                ('regressor',
                 StackingRegressor(cv=5,
                                   estimators=[('ridge', Ridge()),
                                               ('rf',
                                                RandomForestRegressor(random_state=42)),
                                               ('gbr',
                                                GradientBoostingRegressor(learning_rate=0.05,
                                                                          n_estimators=300,
                                                                          random_state=42))],
                                   final_estimator=Ridge(),
                                   passthrough=True))])

In [ ]:
# Оцінка моделі

In [52]:
# Прогнозування на валідаційній вибірці та розрахунок RMSE
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"**Validation RMSE:** {rmse:.4f}")

**Validation RMSE:** 0.1233


In [53]:
# Крос-валідація для більш надійної оцінки
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')
cv_rmse = -np.mean(cv_scores)
print(f"**Cross-Validation RMSE:** {cv_rmse:.4f}")

**Cross-Validation RMSE:** 0.1268


In [56]:
# Прогнозування на тестових даних та формування submission
test_pred = model.predict(test)
test_pred = np.expm1(test_pred)  # Зворотне логарифмічне перетворення
submission = pd.DataFrame({'Id': test_ids, 'SalePrice': test_pred})
submission.to_csv("data/submission.csv", index=False)
print("Файл submission.csv готовий!")
submission

Файл submission.csv готовий!


,Id,SalePrice
0,1461,119706.228360
1,1462,155578.296573
2,1463,182936.933085
3,1464,191869.915555
4,1465,207834.252066
...,...,...
1454,2915,86851.553431
1455,2916,79875.478298
1456,2917,168234.536001
1457,2918,127794.681402
